# sprint14-dnn-framework

# LogisticRegressionの実装

In [120]:
import numpy as np

## 学習用のトレーニングデータを生成します。
ANDゲートでは入力が2次元で出力が1次元となります。 
2つの入力が1のときだけ1を出力し、それ以外は0を出力するので以下のように定義できます。

In [121]:
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

In [122]:
x_train

array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])

In [123]:
y_train

array([[0],
       [0],
       [0],
       [1]])

## 入力データxと正解出力データtを変数tf.placeholder()を使って定義していきます。

In [124]:
x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 1])

In [125]:
x

<tf.Tensor 'Placeholder_10:0' shape=(?, 2) dtype=float32>

In [126]:
t

<tf.Tensor 'Placeholder_11:0' shape=(?, 1) dtype=float32>

第一引数のtf.float32で行列要素の数値のデータ型を指定しています。第二引数の[None,2]で行列の形を指定しています。 
ここで定義されている2はデータの次元を表しています。Noneの部分はデータ数を表す部分です。

今回のANDゲートの場合のデータ数は[0,0],[0,1],[1,0],[1,1]の4つしかないのでNoneの部分を[4,2]としても問題はありません。

しかし、ここで定義したxとtはまだ値が何も入っておらず、ただの入れ物にすぎません。

パラメータの最適化処理の際には、データを一部だけtf.placeholder()に入れて計算することもあります。そのような、データ数が可変の場合に任意の数のデータを入れられるように、一般的にはNoneを使います。


※
https://www.tensorflow.org/api_docs/python/tf/placeholder

## 重みとバイアスは変数tf.Valiable()を使って定義します。

In [127]:
W = tf.Variable(tf.zeros([2, 1]))
b = tf.Variable(tf.zeros([1]))

In [128]:
W

<tf.Variable 'Variable_10:0' shape=(2, 1) dtype=float32_ref>

In [129]:
b

<tf.Variable 'Variable_11:0' shape=(1,) dtype=float32_ref>

ここで、tf.Variable()の中でtf.zeros()という関数を呼び出していますが、変数の初期値を0にしています。つまり2×1行列のすべての要素が0に設定されます。この初期値の状態からWとbの値を学習していくことが今回の目的となります。

※
https://www.tensorflow.org/api_docs/python/tf/Variable

※
https://www.tensorflow.org/api_docs/python/tf/zeros

## 次にモデルの出力yを定義します。 
LogisticRegressionの出力yは活性化関数であるシグモイド関数を使って出力させます。DeepLearning入門1でLogisticRegressionとPerceptronは若干異なると説明しましたが、その違いはここになります。Perceptronの場合には活性化関数にステップ関数(入力が0より大きければ1、それ以外であれば0を出力する関数)を使用します。
では実装してみましょう。普段、関数を使う場合は

In [130]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

のように定義してから使います。しかし、TensorFlowの場合は、

In [131]:
y = tf.nn.sigmoid(tf.matmul(x, W) + b)

と書けば定義しなくても実装することができます。sigmoidの中のtf.matmul(x,W)はxとWの行列の積を表しています。 
別の例として、ソフトマックス関数を使いたい場合は、y = tf.nn.softmax(tf.matmul(x,W) + b)とします。


※
https://www.tensorflow.org/api_docs/python/tf/sigmoid

## モデルの出力を定義した後は、誤差関数を定義します。
今回は交差エントロピー誤差関数を使用します。

In [132]:
cross_entropy = -tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y))

tf.reduce_sum()で行列の要素の総和を計算しています。今回は交差エントロピー誤差関数を使いましたが、もしも二乗和誤差関数を使用したければtf.reduce_sum(tf.square(y - t))と書けば定義できます。tf.square()は行列の要素ごとの二乗を計算します。この2つからわかるようにTensorFlowでは、数式の見た目とほぼ同じように誤差関数を定義することができました。

※
https://www.tensorflow.org/api_docs/python/tf/reduce_sum

※
https://www.tensorflow.org/api_docs/python/tf/square

※
https://www.tensorflow.org/api_docs/python/tf/log

    
## GradientDescent(勾配降下法)を用いてパラメータを最適化

誤差関数は最小化させることに意味がありました。次はGradientDescent(勾配降下法)を用いてパラメータを最適化しましょう。GradientDescentについては次章DeepLearning入門5で詳しく扱いますのでここでは誤差関数を最小化している(正解データと出力データの誤差を最小化している)と言うことだけ理解して読み進めてください。

In [133]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

GradientDescentOptimizer()は引数で学習率を指定しています。学習率についても次章で扱います。

※
https://www.tensorflow.org/api_docs/python/tf/train/GradientDescentOptimizer
    
    
ここまでで学習のための式の定義は終わりです。 

## 正解率の表示
ここで学習後の結果の正解が正しいかどうかの判定と正解率を表示させておきたいと思います。TensorFlowの関数を使って式を定義していきましょう。

In [134]:
correct_prediction = tf.equal(tf.sign(y-0.5),tf.sign(t-0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

1行目で結果が正解かどうか判定しています。一つ一つ見ていきましょう。 

まずtf.equal()は引数に指定された2つの値が等しいかどうかを判定してくれます。返り値はBool値です。

tf.sign()は引数の値が正なら1、0なら0、負なら-1を返します。yが0.5以上かどうかで結果が決まるので、y-0.5とt-0.5の符号を比較しています。

TensorFlowでは値を浮動少数点で扱っているため、出力yの値がぴったり0になることはほぼありえません。 なのでここでは引数の符号が+か-どうかで判定しています。



2行目は正解率を計算するためのコードです。 

tf.reduce_mean()引数に指定された多次元配列の各成分の平均を計算する関数です。 

tf.cast()でBool値を0,1に変換しています。 

つまりここでは正解で1、不正解で0と判定された配列の平均値をとっているので正解率を表していることになります。

※
https://www.tensorflow.org/api_docs/python/tf/equal

※
https://www.tensorflow.org/api_docs/python/tf/sign

※
https://www.tensorflow.org/api_docs/python/tf/reduce_mean

※
https://www.tensorflow.org/api_docs/python/tf/cast

## セッションを準備してパラメータを最適化
ここまでできたら、最後にセッションを準備してパラメータを最適化していきましょう。

TensorFlowでは必ずセッションを定義してパラメータ(重み、バイアス)を計算していきます。簡単なのでコードを見ていきましょう。

In [135]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

まずtf.Session()を変数に格納します。そしてsess.run()でセッションの実行、tf.global_variables_initializer()で上で定義したtf.Variable()の値(重み、バイアス)が初期化されます。

※
https://www.tensorflow.org/api_docs/python/tf/global_variables_initializer

## 最適化の繰り返し回数(エポック数)を指定
セッションを準備したら最適化の繰り返し回数(エポック数)を指定していきます。今回はパラメータ最適化処理を1000回繰り返すことにします。エポック数とその時点での正解率も同時に表示できるようにしておきます。

In [136]:
for epoch in range(1000):
    sess.run(train_step, feed_dict={x:x_train, t:y_train})
    #ここから下は100エポックごとに正解率を表示させています。
    #１００で割った余りがゼロだったら
    if epoch % 100 == 0:
        acc_val = sess.run(
            accuracy, feed_dict={
                x:x_train,
                t:y_train})
        print ('epoch: %d, Accuracy: %f'
                %(epoch, acc_val))

epoch: 0, Accuracy: 0.750000
epoch: 100, Accuracy: 1.000000
epoch: 200, Accuracy: 1.000000
epoch: 300, Accuracy: 1.000000
epoch: 400, Accuracy: 1.000000
epoch: 500, Accuracy: 1.000000
epoch: 600, Accuracy: 1.000000
epoch: 700, Accuracy: 1.000000
epoch: 800, Accuracy: 1.000000
epoch: 900, Accuracy: 1.000000


まず1行目、エポック数を1000回に指定しています。

2行目、sess.run()の引数にtrain_stepを入れることで上で定義したtrain_stepを実行しています。

つまり勾配降下法によって実際に学習させるのがこの部分になります。


この時点で形だけ定義していたplaceholder(xとt)の中にはまだ何も入っていません。

placeholderに値を設定するためにsess.run()のパラメータにfeed_dictというものを使用します。

feed_dict={x:x_train,t:y_train})と書くことで空箱だったxにx_trainの値が入り、tにy_trainの値が入ります。


正解率の表示に関してもcorrect_predictionやaccuracyにまだ値が入っていないためfeed_dictを指定します。


今回はモデルが簡単すぎたため、100回目でもうすでにAccuracyが100%になりました。

モデルが複雑になると100%が出ることはほぼあり得ません。

## 結果確認
では、一旦この表示結果を見てみましょう。

最後に結果を確認してLogisticRegressionの実装を終わりましょう。結果の確認には.eval()を使います。

In [137]:
#学習結果が正しいか確認する
classified = correct_prediction.eval(session=sess, feed_dict ={
    x:x_train, 
    t:y_train
})
#出力yの確認
prob = y.eval(session=sess, feed_dict={
    x:x_train,
    t:y_train
})

必ず引数でsessionを指定しましょう。また、今回もcorrect_predictionとyには値が入っていない状態のためfeed_dictで値を入れましょう。
## 結果は

In [138]:
print(classified)

[[ True]
 [ True]
 [ True]
 [ True]]


In [139]:
print(prob)

[[1.9651421e-04]
 [4.9049832e-02]
 [4.9049832e-02]
 [9.3120378e-01]]



これの見方ですが、eは＊１０という意味で、-04はマイナス４乗の意味になります。

- 1.9に１０のマイナス4乗かけるということは、点を左に2回進めることになるので、0.00019になるので、ほぼ0です。
- 4.9に１０のマイナス2乗かけるということは、点を左に2回進めることになるので、0.049になるので、ほぼ0です。
- 9.3に１０のマイナス1乗かけるということは、点を左に1回進めることになるので、0.93になるので、ほぼ1です。

classifiedの結果は全てTrueで正しく学習されていることがわかります。
probは上からほぼ[0,0,0,1]となっていることがわかります。今回活性化関数に用いたのはシグモイド関数ですので出力yは確率として表示されています。

ANDゲートの正解出力tが[0,0,0,1]となるので混乱しやすいですが、

yはニューロンが発火する(1)か発火しない(0)かを確率で判断しています。なので意味としては上から3つ→1になる確率がほぼ0％、一番下の1つ→93％程度の確率で1になるということを表しています。


## Wとbが学習後どのような値になっているかも見ておきましょう。

In [140]:
print('W:', sess.run(W))
print('b:', sess.run(b))



W: [[5.5699544]
 [5.5699544]]
b: [-8.534579]


Wとbの学習結果を表示させるためには、sess.run()を使えば値を得ることができます。なぜ今回はeval()ではなくsess.run()なのかはtf.Valiable()で定義しているかしていないかです。tf.Valiable()で定義していればsess.run()で取得した値が返ってきます。

# MLPの実装

## XORゲートの学習

ここからはMLPを実装してXORゲートの学習の学習をしていきます。XORゲートは線形分離が不可能です。なので前回学んだLogisticRegressionではXORゲートを学習することができません。ではどうするのか、今回は新たに隠れ層も実装していきます。MLPの実装を通してTensorFlowでの隠れ層の使い方を学んでいきましょう。とは言っても、ここで基本となるのは前回のLogisticRegressionで学んだことです。変数の使い方など基本的な部分は同じです。ではコードを見ていきましょう。

## XORゲートでは2つの入力のどちらかだけ1の場合1を出力します。

In [141]:
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[1],[1],[0]])

## 入力2次元、出力1次元は前回と同じなので

In [142]:
x = tf.placeholder(tf.float32, [None,2])
t = tf.placeholder(tf.float32, [None,1])

## 次は前回と違う点、隠れ層の設定をしていきます。

In [143]:
W1 = tf.Variable(tf.truncated_normal([2,2]))
b1 = tf.Variable(tf.zeros([2]))
h = tf.nn.sigmoid(tf.matmul(x,W1) + b1)

W1は一層目の重みを表しています。W1の引数であるtf.truncated_normal()は切断正規分布に基づいたデータを生成させています。

切断正規分布は、ここでは誤差逆伝播法をうまく作用させるために乱数で初期化しているということだけ押さえておきましょう。

LogisticRegressionのときにようにパラメータをすべて0で初期化してしまうと誤差逆伝播法がうまく作用されないことがあります。

（もちろんtruncated_normal()ではなく、random_normal()でも作用します。）

その中の[2,2]は[隠れ層の入力次元,隠れ層のノードの数]を指定しています。 

b1は入力層から隠れ層に入力されるバイアスです。

ここで指定されている[2]も隠れ層のノード数を指定しています。

そしてhが隠れ層を表します。層を追加することの利点は、非線形関数を用いることで、線形関数では対処できない問題に対して対処できるように値を変換することでした。ここではシグモイド関数を使って隠れ層に入ってきた重み付き和を変換します。

ここでは隠れ層は一層しか導入しませんが、さらに層を追加したい場合も同じように定義していけば実装することができます。


## モデルの出力

In [144]:
W2 = tf.Variable(tf.truncated_normal([2,1]))
b2 = tf.Variable(tf.zeros([1]))
y = tf.nn.sigmoid(tf.matmul(h, W2) + b2)

コードの意味は先程の隠れ層と同じです。コードを見てみるとノードの数が変化していることがわかります。

今定義しているのは出力層なので、これは出力層のノードの数が1つであることを意味します。

誤差関数はPerceptronと同様に交差エントロピー誤差関数を使い、勾配降下法の実装も前回と同じです。

In [145]:
cross_entropy = -tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
correct_prediction = tf.equal(tf.sign(y-0.5),tf.sign(t-0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## 学習過程の実装も前回と同様

In [146]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(6000):
    sess.run(train_step,feed_dict={
        x:x_train,
        t:y_train
    })

    if epoch % 100 == 0:
        acc_val = sess.run(
            accuracy,feed_dict={
                x:x_train,
                t:y_train})
        print ('epoch: %d, Accuracy: %f'
               %(epoch, acc_val))

classified = correct_prediction.eval(session=sess, feed_dict={
    x:x_train,
    t:y_train
})

prob = y.eval(session=sess,feed_dict={
    x:x_train,
})

epoch: 0, Accuracy: 0.500000
epoch: 100, Accuracy: 0.500000
epoch: 200, Accuracy: 0.500000
epoch: 300, Accuracy: 0.500000
epoch: 400, Accuracy: 0.500000
epoch: 500, Accuracy: 0.500000
epoch: 600, Accuracy: 0.500000
epoch: 700, Accuracy: 0.750000
epoch: 800, Accuracy: 0.750000
epoch: 900, Accuracy: 0.750000
epoch: 1000, Accuracy: 0.750000
epoch: 1100, Accuracy: 0.750000
epoch: 1200, Accuracy: 0.750000
epoch: 1300, Accuracy: 0.750000
epoch: 1400, Accuracy: 0.750000
epoch: 1500, Accuracy: 0.500000
epoch: 1600, Accuracy: 0.500000
epoch: 1700, Accuracy: 0.500000
epoch: 1800, Accuracy: 0.500000
epoch: 1900, Accuracy: 0.500000
epoch: 2000, Accuracy: 0.500000
epoch: 2100, Accuracy: 0.500000
epoch: 2200, Accuracy: 0.500000
epoch: 2300, Accuracy: 0.500000
epoch: 2400, Accuracy: 0.500000
epoch: 2500, Accuracy: 0.500000
epoch: 2600, Accuracy: 0.500000
epoch: 2700, Accuracy: 0.500000
epoch: 2800, Accuracy: 0.500000
epoch: 2900, Accuracy: 0.500000
epoch: 3000, Accuracy: 0.500000
epoch: 3100, Accurac

In [147]:
print(classified)

[[ True]
 [False]
 [ True]
 [False]]


In [148]:
print(prob)

[[0.00266801]
 [0.49924862]
 [0.99617094]
 [0.5019172 ]]


In [149]:
print('W1:', sess.run(W1))
print('b1:', sess.run(b1))
print('W2:', sess.run(W2))
print('b2:', sess.run(b2))

W1: [[-5.559029   3.3968854]
 [-8.926386  -8.54109  ]]
b1: [ 1.5105188 -1.9390633]
W2: [[-8.312973]
 [ 7.029182]]
b2: [0.00179539]


# GradientDescentとは

ニューラルネットワークの学習では、誤差関数が最小になるように最適なパラメータ(重み、バイアス)を探索します。このパラメータの探索方法の一つにGradientDescent(勾配降下法)と呼ばれる手法があります。これは名前の通り誤差関数のGradient(勾配)を利用して最適化パラメータを見つけ出す手法です。勾配は誤差関数を微分することによって求めることができます。

一変数の考え方は簡単です。
下に凸の二次関数の場合、与えられた初期値での微分が正の場合左へ、負の場合右へ進みます。



勾配降下法は方程式から解を直接求めるのではなく、イメージ的には上図のように、少しずつ点を動かしながら最小値を探し出していく方法です。

※GradientDescentは関数の最小値を見つける手法です。上に凸の関数の最大値を探す手法はGradientAscentと呼ばれます。ニューラルネットワークで使われる誤差関数は下に凸の関数であるため、GradientDescentを使います。


# GradientDescentの実装

In [150]:
tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

<tf.Operation 'GradientDescent_6' type=NoOp>

前章までではあまり触れなかったのでここでさらに詳しく見て行きましょう。
まずはtf.train.GradientDescentOptimizerの次に出てくる(0.1)についてです。これが上式で出てきた学習率を指定する部分になります。学習率はハイパーパラメータなのでした。よってこの学習率をどのくらいにするか自分で決める必要があります。一般的に0.1,0.01,0.001など小さい値を用います。この値が小さすぎても大きすぎても最適なパラメータをうまく見つけられなさそうなことは直感的にわかると思います。ハイパーパラメータに関しては、自分で値を変更していき、学習が正しく、精度よく行われているか確認しながら学習を進めていく必要があります。
そして.minimize()で最小化させる関数を指定しています。
このコードは、GradientDescentを使って誤差関数(cross_entropy)を最小化させるということを行なっていることがなんとなくわかると思います。

※
https://www.tensorflow.org/api_docs/python/tf/train/GradientDescentOptimizer

# Backpropagationとは

backpropagation(誤差逆伝播法)とは、多数のパラメータを持つ関数の各パラメータの微分係数を高速に計算するためのアルゴリズムです。
機械学習で用いるGradientDiscentでは，パラメータの更新に誤差関数の勾配(偏微分係数)を計算する必要がありました。通常、コンピュータで係数を計算する場合、数値微分というアルゴリズムで計算を行います。
ただ、NeuralNetworkのパラメータは非常に多くなります。例えば、特徴量が100個あった場合、

$y = w_0 + w_1 x_1 + \dots + w_{100} x_{100}$

において、誤差関数の偏微分係数を計算していくという作業になるので、大変計算時間がかかります。
そこで考案された高速な手法がbackpropagationという手法になります。

In [151]:
tf.summary.FileWriter('XOR', sess.graph)

実行が終わったら、ターミナルを開いて以下を打ち込みましょう。

$ tensorboard --logdir=/ディレクトリの絶対パス/XOR

ディレクトリのパスはFinderからターミナルにXORディレクトリをドラッグアンドドロップすれば取得できます。
起動できたら、localhost:6006にアクセスすれば、TensorBoardが表示されます。
    
Chromeで起動するのが望ましいです。Safariだとうまく起動しないことがあります。(ブラウザにlocalhost:6006と入れれば出てきます。)
可視化することができました。出てきたグラフを見てみましょう。

グラフは表示できていますが、何が何を表しているのか全然わからないと思います。

http://hirokisuzaki.local:6006/#graphs&run=.

In [152]:
x = tf.placeholder(tf.float32, [None,2], name='x')
t = tf.placeholder(tf.float32, [None,1], name='t')

W1 = tf.Variable(tf.truncated_normal([2,2]), name='W1')
b1 = tf.Variable(tf.zeros([2]), name='b1')
h = tf.nn.sigmoid(tf.matmul(x,W1) + b1, name='h')

W2 = tf.Variable(tf.truncated_normal([2,1]), name='W2')
b2 = tf.Variable(tf.zeros([1]), name='b2')
y = tf.nn.sigmoid(tf.matmul(h, W2) + b2, name='y')

In [153]:
with tf.name_scope('cross_entropy'):
  cross_entropy = -tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y))

with tf.name_scope('train'):
  train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

with tf.name_scope('accuracy'):
  correct_prediction = tf.equal(tf.sign(y-0.5),tf.sign(t-0.5))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.name_scope()は、誤差の計算や正解率の計算など、計算処理をひとまとめにして名前をつけて表示させたい時に使います。

※
https://www.tensorflow.org/api_docs/python/tf/name_scope

としましょう。この結果はTensorBoardのSCALARSというところに記録されます。sess.run()の中にmergedを入れて実行します。今回はエポックごとにcross_entropyとaccuracyの値も得たいのでこの2つも入れます。.最後.add_summary()によって、ここで取得したものをディレクトリに書き込んでいきます。

In [159]:
tf.summary.histogram('weights_h', W1)
tf.summary.histogram('biases_h', b1)
tf.summary.histogram('weights_out', W2)
tf.summary.histogram('biases_out', b2)

<tf.Tensor 'biases_out_2:0' shape=() dtype=string>

In [160]:
tf.summary.FileWriter('XOR', sess.graph)